In [25]:
#-- Modules --#
import pandas as pd
import requests
import datetime
import logging
import warnings
import os
warnings.filterwarnings('ignore')
#-- logging --#
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('client_coes.log')
    ]
)

In [26]:
logger = logging.getLogger(__name__)
#-- Main code --#
def clientCoes(code:int, service, startDate, 
               endDate, *args, **kwargs):
    #-- Settings --#
    startDate = startDate.strftime("%Y-%m-%d")
    endDate = endDate.strftime("%Y-%m-%d")
    api = f'api/Mediciones?lectcodi={code}&fechaIni={startDate}&fechaFin={endDate}'
    url = f'{service}{api}'
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        obj = pd.DataFrame(response.json())
        print(url)
        if not obj.empty:
            queryList = ['Cupis', 'Dunas', 'Huambos', 'Marcona', 'Lomitas', 
                         'Talara', 'Hermanas', 'Wayra', 'C.E.' ]
            obj = obj[obj['NombreEquipo'].str.contains('|'.join(queryList), case=False, regex=True)].copy()
            queryList = ['TV', 'C.H.', 'C.S.']
            obj = obj[~obj['NombreEquipo'].str.contains('|'.join(queryList), case=False, regex=True)].copy()
        logger.info(f"Successfully retrieved data for code {code}")
        return obj.iloc[:,:-48]
    
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed for code {code}: {e}")
        return pd.DataFrame()
    except Exception as e:
        logger.error(f"Unexpected error processing code {code}: {e}")
        return pd.DataFrame()

def getFormatData(dataset: pd.DataFrame, *args, **kwargs):
    try:
        idxNames = [c for c in dataset.columns if not any(f'h{w}' in c for w in range(1,49))]
        print("COLUMNS RAW:", [repr(c) for c in dataset.columns])
        print("COLUMNS STRIP:", [repr(str(c).strip()) for c in dataset.columns])
        data = dataset.melt(id_vars=idxNames, value_name='power', var_name='hour').sort_values(['NombreEquipo','Medifecha'])
        data['Medifecha'] = pd.to_datetime(data['Medifecha'], format="%Y-%m-%dT00:00:00")
        data['hour_timedelta'] = pd.to_timedelta(data['hour'].str.replace('h', '').astype(int) * 30, unit='m')
        data['date'] = data['Medifecha'] + data['hour_timedelta']
        data = data[['date']+idxNames+['power']]
        data = data.sort_values(['NombreEquipo','date'])
        #-- Punta Lomitas --#
        puntaLomitas = data[data['NombreEquipo'].str.contains('PUN', case=False, regex=True)]
        puntaLomitas = puntaLomitas.groupby(['date']).agg({'NombreEmpresa':'last', 'Tipoinfoabrev':'last', 'power':'sum'}).reset_index()
        puntaLomitas['name'] = 'W.F. Punta Lomitas'
        #-- All except Punta Lomitas --#
        epuntaLomitas = data[~data['NombreEquipo'].str.contains('|'.join(['PUN','paita']), 
                                                                case=False, regex=True)]
        repNames = {
            'C.E. CUPISNIQUE':'W.F. Cupisnique',
            'C.E. DUNA':'W.F. Duna',
            'C.E. HUAMBOS':'W.F. Huambos',
            'C.E. MARCONA':'W.F. Marcona',
            'C.E. SAN JUAN':'W.F. San Juan',
            'C.E. TALARA':'W.F. Talara',
            'C.E. TRES HERMANAS':'W.F. Tres Hermanas',
            'C.E. WAYRA EXTENSION':'W.F. Wayra Ext',
            'C.E. WAYRA I':'W.F. Wayra I'
        }
        epuntaLomitas.loc[:,'NombreEquipo'] = epuntaLomitas['NombreEquipo'].replace(repNames)
        epuntaLomitas = epuntaLomitas.rename({'NombreEquipo':'name'},axis=1)
        epuntaLomitas = epuntaLomitas[puntaLomitas.columns]
        #-- Merged Both Datasets --#
        data = pd.concat([puntaLomitas, epuntaLomitas], ignore_index=True)
        logger.info("Data formatting completed successfully")
        return data
    except Exception as e:
        logger.error(f"Error in data formatting: {e}")
        raise


In [27]:
#-- Run code --#
if __name__ == '__main__':
    try:
        service = 'https://appserver.coes.org.pe/waMediciones/'
        endDate = datetime.datetime.now()
        startDate = endDate - datetime.timedelta(days=5)
        logger.info(f"Starting data retrieval from {startDate.strftime('%Y-%b-%d')} to {endDate.strftime('%Y-%b-%d')}")
        #-- Create Dataset Dir --#
        parquet_path = '../dataset/currentGen.parquet'
        os.makedirs(os.path.dirname(parquet_path), exist_ok=True)
        #-- Iterations through codes --#
        container = []
        for code in range(0, 4):
            logger.debug(f"Processing code {code}")
            result = clientCoes(code, service, startDate, endDate)
            if not result.empty:
                container.append(result)
        if container:
            dataset = pd.concat(container, ignore_index=False)
            #-- Discard duplicates --#
            dataset = dataset.groupby("NombreEquipo", group_keys=False)\
                           .apply(lambda x: x.drop_duplicates(subset=['Medifecha'], keep='last'))\
                           .reset_index()
            dropColumns = ['TensionEquipo', 'CodigoUbicacion', 'NombreUbicacion', 'index', '$id']
            dataset = dataset.drop(dropColumns, axis=1, errors='ignore')
            # #-- Set data format --#
            # dataset = getFormatData(dataset)
            # logger.info(f"Coes Dataset was retrieved! Shape: {dataset.shape}")
            # #-- Export as parquet --#
            # dataset.to_parquet(parquet_path)
            # logger.info(f"Coes Dataset was stored!")
        else:
            logger.warning("No data was retrieved from any of the codes")
    except Exception as e:
        logger.exception("An unexpected error occurred during execution")

2026-01-22 12:22:28,417 - INFO - Starting data retrieval from 2026-Jan-17 to 2026-Jan-22
2026-01-22 12:22:29,013 - INFO - Successfully retrieved data for code 0


https://appserver.coes.org.pe/waMediciones/api/Mediciones?lectcodi=0&fechaIni=2026-01-17&fechaFin=2026-01-22


2026-01-22 12:22:29,578 - INFO - Successfully retrieved data for code 1


https://appserver.coes.org.pe/waMediciones/api/Mediciones?lectcodi=1&fechaIni=2026-01-17&fechaFin=2026-01-22


2026-01-22 12:22:30,374 - INFO - Successfully retrieved data for code 2


https://appserver.coes.org.pe/waMediciones/api/Mediciones?lectcodi=2&fechaIni=2026-01-17&fechaFin=2026-01-22


2026-01-22 12:22:33,315 - INFO - Successfully retrieved data for code 3


https://appserver.coes.org.pe/waMediciones/api/Mediciones?lectcodi=3&fechaIni=2026-01-17&fechaFin=2026-01-22


In [4]:
dataset

,Medifecha,Ptomedicodi,NombreEquipo,CodigoEquipo,CodigoEmpresa,NombreEmpresa,Tipoinfoabrev,h1,h2,h3,...,h39,h40,h41,h42,h43,h44,h45,h46,h47,h48
0,2026-01-17T00:00:00,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,33.498,33.498,32.1490,...,55.7520,55.7520,48.4605,48.4605,41.2055,41.2055,35.3685,35.3685,30.2265,30.2265
1,2026-01-18T00:00:00,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,25.046,25.046,21.9325,...,51.7245,51.7245,44.8580,44.8580,37.9915,37.9915,32.8765,32.8765,28.8120,28.8120
2,2026-01-19T00:00:00,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,24.748,24.748,22.3600,...,48.0455,48.0455,41.4010,41.4010,34.7565,34.7565,30.4370,30.4370,27.5125,27.5125
3,2026-01-20T00:00:00,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,24.588,24.588,22.9640,...,46.4930,46.4930,39.4535,39.4535,32.5925,32.5925,27.5800,27.5800,23.6765,23.6765
4,2026-01-21T00:00:00,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,19.773,19.773,17.6455,...,43.2665,43.2665,38.3870,38.3870,33.5080,33.5080,28.7470,28.7470,24.0565,24.0565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2026-01-18T00:00:00,770,C.E. TRES HERMANAS,15160,11218,PARQUE EOLICO TRES HERMANAS S.A.C.,MW,35.439,35.439,29.0175,...,45.3945,45.3945,42.6365,42.6365,39.8790,39.8790,34.8420,34.8420,28.4380,28.4380
74,2026-01-19T00:00:00,770,C.E. TRES HERMANAS,15160,11218,PARQUE EOLICO TRES HERMANAS S.A.C.,MW,22.035,22.035,18.5625,...,41.1780,41.1780,37.2310,37.2310,33.2835,33.2835,29.5585,29.5585,25.9680,25.9680
75,2026-01-20T00:00:00,770,C.E. TRES HERMANAS,15160,11218,PARQUE EOLICO TRES HERMANAS S.A.C.,MW,22.377,22.377,19.6300,...,28.3680,28.3680,26.1935,26.1935,24.3165,24.3165,21.6020,21.6020,18.3850,18.3850
76,2026-01-21T00:00:00,770,C.E. TRES HERMANAS,15160,11218,PARQUE EOLICO TRES HERMANAS S.A.C.,MW,15.168,15.168,13.2220,...,24.7200,24.7200,21.7230,21.7230,18.7260,18.7260,16.0975,16.0975,13.6900,13.6900


In [23]:
idxNames = [c for c in dataset.columns if not any(f'h{w}' in c for w in range(1,49))]
print("COLUMNS RAW:", [repr(c) for c in dataset.columns])
print("COLUMNS STRIP:", [repr(str(c).strip()) for c in dataset.columns])
data = dataset.melt(id_vars=idxNames, value_name='power', var_name='hour').sort_values(['NombreEquipo','Medifecha'])
data['Medifecha'] = pd.to_datetime(data['Medifecha'], format="%Y-%m-%dT00:00:00")
data['hour_timedelta'] = pd.to_timedelta(data['hour'].str.replace('h', '').astype(int) * 30, unit='m')
data['date'] = data['Medifecha'] + data['hour_timedelta']
data = data[['date']+idxNames+['power']]
data = data.sort_values(['NombreEquipo','date'])
#-- Punta Lomitas --#
puntaLomitas = data[data['NombreEquipo'].str.contains('PUN', case=False, regex=True)]
puntaLomitas = puntaLomitas.groupby(['date']).agg({'NombreEmpresa':'last', 'Tipoinfoabrev':'last', 'power':'sum'}).reset_index()
puntaLomitas['name'] = 'W.F. Punta Lomitas'
#-- All except Punta Lomitas --#
epuntaLomitas = data[~data['NombreEquipo'].str.contains('|'.join(['PUN','paita']), 
                                                        case=False, regex=True)]
repNames = {
    'C.E. CUPISNIQUE':'W.F. Cupisnique',
    'C.E. DUNA':'W.F. Duna',
    'C.E. HUAMBOS':'W.F. Huambos',
    'C.E. MARCONA':'W.F. Marcona',
    'C.E. SAN JUAN':'W.F. San Juan',
    'C.E. TALARA':'W.F. Talara',
    'C.E. TRES HERMANAS':'W.F. Tres Hermanas',
    'C.E. WAYRA EXTENSION':'W.F. Wayra Ext',
    'C.E. WAYRA I':'W.F. Wayra I'
}
epuntaLomitas.loc[:,'NombreEquipo'] = epuntaLomitas['NombreEquipo'].replace(repNames)
epuntaLomitas = epuntaLomitas.rename({'NombreEquipo':'name'},axis=1)
epuntaLomitas = epuntaLomitas[puntaLomitas.columns]
#-- Merged Both Datasets --#
data = pd.concat([puntaLomitas, epuntaLomitas], ignore_index=True)
logger.info("Data formatting completed successfully")

2026-01-22 11:54:39,728 - INFO - Data formatting completed successfully


COLUMNS RAW: ["'Medifecha'", "'Ptomedicodi'", "'NombreEquipo'", "'CodigoEquipo'", "'CodigoEmpresa'", "'NombreEmpresa'", "'Tipoinfoabrev'", "'h1'", "'h2'", "'h3'", "'h4'", "'h5'", "'h6'", "'h7'", "'h8'", "'h9'", "'h10'", "'h11'", "'h12'", "'h13'", "'h14'", "'h15'", "'h16'", "'h17'", "'h18'", "'h19'", "'h20'", "'h21'", "'h22'", "'h23'", "'h24'", "'h25'", "'h26'", "'h27'", "'h28'", "'h29'", "'h30'", "'h31'", "'h32'", "'h33'", "'h34'", "'h35'", "'h36'", "'h37'", "'h38'", "'h39'", "'h40'", "'h41'", "'h42'", "'h43'", "'h44'", "'h45'", "'h46'", "'h47'", "'h48'"]
COLUMNS STRIP: ["'Medifecha'", "'Ptomedicodi'", "'NombreEquipo'", "'CodigoEquipo'", "'CodigoEmpresa'", "'NombreEmpresa'", "'Tipoinfoabrev'", "'h1'", "'h2'", "'h3'", "'h4'", "'h5'", "'h6'", "'h7'", "'h8'", "'h9'", "'h10'", "'h11'", "'h12'", "'h13'", "'h14'", "'h15'", "'h16'", "'h17'", "'h18'", "'h19'", "'h20'", "'h21'", "'h22'", "'h23'", "'h24'", "'h25'", "'h26'", "'h27'", "'h28'", "'h29'", "'h30'", "'h31'", "'h32'", "'h33'", "'h34'", 

In [24]:
data

,date,NombreEmpresa,Tipoinfoabrev,power,name
0,2026-01-17 00:30:00,ENGIE ENERGIA PERU S.A.A.,MW,91.3520,W.F. Punta Lomitas
1,2026-01-17 01:00:00,ENGIE ENERGIA PERU S.A.A.,MW,91.3520,W.F. Punta Lomitas
2,2026-01-17 01:30:00,ENGIE ENERGIA PERU S.A.A.,MW,90.8245,W.F. Punta Lomitas
3,2026-01-17 02:00:00,ENGIE ENERGIA PERU S.A.A.,MW,90.8245,W.F. Punta Lomitas
4,2026-01-17 02:30:00,ENGIE ENERGIA PERU S.A.A.,MW,93.7885,W.F. Punta Lomitas
...,...,...,...,...,...
2875,2026-01-22 22:00:00,ORYGEN PERU S.A.A.,MW,72.5970,W.F. Wayra I
2876,2026-01-22 22:30:00,ORYGEN PERU S.A.A.,MW,57.0050,W.F. Wayra I
2877,2026-01-22 23:00:00,ORYGEN PERU S.A.A.,MW,57.0050,W.F. Wayra I
2878,2026-01-22 23:30:00,ORYGEN PERU S.A.A.,MW,42.5130,W.F. Wayra I


In [18]:
puntaLomitas['NombreEquipo'].unique()

array(['C.E. PUN LOM_EXP-BL1', 'C.E. PUN LOM_EXP-BL2',
       'C.E. PUNTA LOMITAS-BL1', 'C.E. PUNTA LOMITAS-BL2'], dtype=object)

In [10]:
data

,date,Medifecha,Ptomedicodi,NombreEquipo,CodigoEquipo,CodigoEmpresa,NombreEmpresa,Tipoinfoabrev,power
0,2026-01-17 00:30:00,2026-01-17,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,33.4980
78,2026-01-17 01:00:00,2026-01-17,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,33.4980
156,2026-01-17 01:30:00,2026-01-17,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,32.1490
234,2026-01-17 02:00:00,2026-01-17,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,32.1490
312,2026-01-17 02:30:00,2026-01-17,256,C.E. CUPISNIQUE,14407,10552,ENERGÍA EÓLICA S.A.,MW,30.3615
...,...,...,...,...,...,...,...,...,...
3413,2026-01-22 22:00:00,2026-01-22,2161,C.E. WAYRA I,18306,15259,ORYGEN PERU S.A.A.,MW,72.5970
3491,2026-01-22 22:30:00,2026-01-22,2161,C.E. WAYRA I,18306,15259,ORYGEN PERU S.A.A.,MW,57.0050
3569,2026-01-22 23:00:00,2026-01-22,2161,C.E. WAYRA I,18306,15259,ORYGEN PERU S.A.A.,MW,57.0050
3647,2026-01-22 23:30:00,2026-01-22,2161,C.E. WAYRA I,18306,15259,ORYGEN PERU S.A.A.,MW,42.5130
